Epoch 1/200


c:\Users\a\anaconda3\envs\jungwoo\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.5469 - mae: 0.5449 - val_loss: 0.4738 - val_mae: 0.5369
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.5139 - mae: 0.5197 - val_loss: 0.4285 - val_mae: 0.4931
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.4656 - mae: 0.4844 - val_loss: 0.3805 - val_mae: 0.4417
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.4290 - mae: 0.4455 - val_loss: 0.3308 - val_mae: 0.3805
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.3904 - mae: 0.4081 - val_loss: 0.2826 - val_mae: 0.3079
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.3524 - mae: 0.3785 - val_loss: 0.2430 - val_mae: 0.2302
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.3491 - mae: 0.3782 - val_loss: 0.2194 - val_mae: 0.1795
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.3023 - mae: 0.3334 - val_loss: 0.2043 - val_mae: 0.1617
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.2892 - mae: 0.3077 - val_l

c:\Users\a\anaconda3\envs\jungwoo\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\a\anaconda3\envs\jungwoo\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Glyph 52264 (\N{HANGUL SYLLABLE CA}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\a\anaconda3\envs\jungwoo\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Glyph 48512 (\N{HANGUL SYLLABLE BU}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\a\anaconda3\envs\jungwoo\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Glyph 53552 (\N{HANGUL SYLLABLE TEO}) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\a\anaconda3\envs\jungwoo\Lib\site-packages\IPython\core\pylabtools.py:170: UserWarning: Glyph 45796 (\N{HANGUL SYLLABLE DA}) missing from font(s) Deja

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 불러오기
yield_data = pd.read_csv('전북군산생산량_변환.csv')
env_data = pd.read_csv('전북군산환경데이터_변환.csv')

# 2. 날짜 데이터를 변환하고 주차 계산
env_data['measDtStr'] = pd.to_datetime(env_data['measDtStr'], format='%Y%m%d%H%M%S')
env_data['연도'] = env_data['measDtStr'].dt.year
env_data['주차'] = env_data['measDtStr'].dt.isocalendar().week

# 3. 적산 계산을 위한 데이터 그룹화 (주차별로 누적 합계 계산)
env_weekly_cumulative = env_data.groupby(['연도', '주차']).agg({
    'inTp': 'sum',     # 온도 적산
    'inHd': 'sum',     # 습도 적산
    'inCo2': 'sum',    # CO2 적산
    'outTp': 'sum'     # 추가된 outTp 적산
}).reset_index()

# 4. 생산량 데이터와 병합
yield_data = yield_data.rename(columns={'item.frmhsId': 'frmhsId'})
merged_data = pd.merge(
    env_weekly_cumulative, 
    yield_data[['연도', '주차', '생산량']],
    on=['연도', '주차'],
    how='inner').dropna(subset=['생산량'])

# 5. 특성과 타겟 데이터 분리
X = merged_data[['inTp', 'inHd', 'inCo2', 'outTp']].values  # 입력 변수 (온도, 습도, CO2, 추가된 outTp 적산값)
y = merged_data['생산량'].values  # 타겟 변수 (생산량)

# 6. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 타겟 데이터 스케일링
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 7. 시계열 데이터 준비
timesteps = 12  # 12주 간 데이터를 사용
forecast_weeks = 1  # 1주차 예측

# 시퀀스를 생성
X_sequences = []
y_sequences = []
for i in range(len(X_scaled) - timesteps - forecast_weeks + 1):
    X_sequences.append(X_scaled[i:i+timesteps])
    y_sequences.append(y_scaled[i+timesteps:i+timesteps+forecast_weeks].flatten())

# 시퀀스를 넘파이 배열로 변환
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# 8. 학습 데이터와 테스트 데이터로 분할 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# 9. LSTM 모델 구축
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dropout(0.3),  # 과적합 방지를 위한 드롭아웃
    tf.keras.layers.LSTM(100, activation='relu', return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),  # 과적합 방지를 위한 드롭아웃
    tf.keras.layers.GRU(50, activation='relu'),  # GRU 층 추가
    tf.keras.layers.Dropout(0.3),  # 과적합 방지를 위한 드롭아웃
    tf.keras.layers.Dense(forecast_weeks)  # 예측할 주차 수 만큼의 출력 뉴런
])

# RMSprop 최적화 알고리즘으로 모델 컴파일
optimizer = RMSprop(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=32, callbacks=[early_stopping])

# 전체 데이터로 시퀀스 생성
X_full_sequences = []
for i in range(len(X_scaled) - timesteps + 1):
    X_full_sequences.append(X_scaled[i:i+timesteps])
X_full_sequences = np.array(X_full_sequences)

# 전체 데이터로 예측 수행
full_predictions = model.predict(X_full_sequences)
full_predictions_inverse = scaler_y.inverse_transform(full_predictions)

# 주차별 예측된 생산량 출력 (47주차부터 다음 해 24주차까지)
weeks_extended_full = np.concatenate([np.arange(47, 53), np.arange(1, 25)])  # 47주차부터 다음 해 24주차까지
filtered_predictions = full_predictions_inverse[:len(weeks_extended_full)]  # 주차 범위에 맞춰 예측값 슬라이싱

# 1. 데이터 로드
yield_data = pd.read_csv('전북군산생산량_변환.csv')

# 2. 47주차부터 다음 해 24주차까지 필터링
filtered_yield_data = yield_data[(yield_data['주차'] >= 47) | (yield_data['주차'] <= 24)]
filtered_yield_data = filtered_yield_data.sort_values(by=['연도', '주차'])

# 4. 각 주차에 대응하는 생산량 데이터 가져오기
actual_production = filtered_yield_data['생산량'].values[:len(weeks_extended_full)]  # 주차에 맞춰 데이터 슬라이싱

# 그래프 시각화 (필요시)
plt.figure(figsize=(14, 7))
plt.plot(np.arange(len(weeks_extended_full)), filtered_predictions.flatten(), label='Predicted Production', color='red', linewidth=2)
plt.plot(np.arange(len(weeks_extended_full)), actual_production, label='Actual Production', color='blue', linewidth=2)
plt.xticks(ticks=np.arange(len(weeks_extended_full)), labels=weeks_extended_full)
plt.title('Predicted vs Actual Weekly Tomato Production (47주차부터 다음 해 24주차까지)')
plt.xlabel('Week')
plt.ylabel('Production')
plt.legend()
plt.grid(True)
plt.show()


In [4]:
# 5. 실제 생산량 및 예측된 생산량을 모두 출력
print("주차별 실제 생산량 및 예측된 생산량:")
for i in range(len(weeks_extended_full)):
    print(f"{weeks_extended_full[i]}주차 실제 생산량 : {actual_production[i]}, 예측 생산량 : {filtered_predictions.flatten()[i]}")

주차별 실제 생산량 및 예측된 생산량:
47주차 실제 생산량 : 2368, 예측 생산량 : 3939.087646484375
48주차 실제 생산량 : 6804, 예측 생산량 : 7140.09228515625
49주차 실제 생산량 : 9888, 예측 생산량 : 8265.2705078125
50주차 실제 생산량 : 8208, 예측 생산량 : 9810.55859375
51주차 실제 생산량 : 8628, 예측 생산량 : 10120.134765625
52주차 실제 생산량 : 8671, 예측 생산량 : 9652.4345703125
1주차 실제 생산량 : 9792, 예측 생산량 : 8097.0322265625
2주차 실제 생산량 : 6540, 예측 생산량 : 6593.8095703125
3주차 실제 생산량 : 5928, 예측 생산량 : 5826.78076171875
4주차 실제 생산량 : 5448, 예측 생산량 : 5381.5
5주차 실제 생산량 : 5160, 예측 생산량 : 5686.99560546875
6주차 실제 생산량 : 2736, 예측 생산량 : 5864.3955078125
7주차 실제 생산량 : 4104, 예측 생산량 : 6379.7470703125
8주차 실제 생산량 : 7068, 예측 생산량 : 6465.00439453125
9주차 실제 생산량 : 6984, 예측 생산량 : 6587.5390625
10주차 실제 생산량 : 7560, 예측 생산량 : 6754.2802734375
11주차 실제 생산량 : 10272, 예측 생산량 : 6985.8994140625
12주차 실제 생산량 : 8640, 예측 생산량 : 7548.33154296875
13주차 실제 생산량 : 7008, 예측 생산량 : 8225.3095703125
14주차 실제 생산량 : 7812, 예측 생산량 : 8821.0927734375
15주차 실제 생산량 : 8316, 예측 생산량 : 9349.248046875
16주차 실제 생산량 : 8568, 예측 생산량 : 10000.626953125
17주차